In [1]:
import tensorflow as tf
import pickle
import pandas as pd
import datetime
import numpy as np

2023-12-03 16:25:48.829479: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-03 16:25:49.124303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 16:25:49.124366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 16:25:49.140428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 16:25:49.174770: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import os
Fd='/glade/derecho/scratch/yizhu/comp576/ORIENT'
os.chdir(Fd)
print(os.getcwd())
print(os.listdir())

/glade/derecho/scratch/yizhu/comp576/ORIENT
['rbspb_ED_MLAT_OP77Q_intxt_5min', 'unix_time_5min', 'rbspa_eflux_OP77Q_intxt_5min', 'mageis_ch14_model_sav.h5', 'rbspb_eflux_OP77Q_intxt_5min', 'mageis_ch11_input_avg.npy', 'rbspa_ED_MLT_OP77Q_intxt_5min', 'rbspa_MLAT_OP77Q_intxt_5min', 'mageis_ch16_input_std.npy', 'ch11_data.pkl', 'mageis_ch3_input_std.npy', 'rbspa_Lm_eq_OP77Q_intxt_5min', 'mageis_ch14_input_std.npy', 'rbspb_ED_R_OP77Q_intxt_5min', 'rbspb_Lm_eq_OP77Q_intxt_5min', 'mageis_ch3_model_sav.h5', 'mageis_ch3_input_avg.npy', 'rbspa_ED_R_OP77Q_intxt_5min', 'ch3_data2.pkl', 'mageis_ch16_model_sav.h5', 'ch3_data3.pkl', 'rbspb_ED_MLT_OP77Q_intxt_5min', 'mageis_ch11_input_std.npy', 'rbspb_MLAT_OP77Q_intxt_5min', 'ch3_data.pkl', '.gitattributes', 'mageis_ch11_model_sav.h5', 'rbspa_ED_MLAT_OP77Q_intxt_5min', 'ch3_data1.pkl', 'mageis_ch16_input_avg.npy', 'mageis_ch14_input_avg.npy']


# get data

In [3]:
with open('ch11_data.pkl','rb') as f:
  data=pickle.load(f)

In [4]:
# remove nan
data=data.dropna()

In [5]:
# remove mean and std
mean=np.load('mageis_ch11_input_avg.npy')
std=np.load('mageis_ch11_input_std.npy')
# remove mean and std
data.iloc[:,2:-1]=(data.iloc[:,2:-1]-mean)/std

In [6]:
# log10 to eflux+1
data['eflux']=data['eflux'].apply(lambda x: np.log10(x+1))

In [7]:
# date from unix time to datetime
data['date']=pd.to_datetime(data['unix_time'],unit='s')

In [8]:
# 3 parts
# split the data into 3 part
# part1 2018-03-01 to 2019-04-01
# start and end time
start1 = datetime.datetime(2018, 3, 1)
end1 = datetime.datetime(2019, 4, 1)
data1=data.loc[(data['date']>=start1) & (data['date']<end1)]

# part2 2017-02-23 to 2017-02-25
start2 = datetime.datetime(2017, 2, 23)
end2 = datetime.datetime(2017, 2, 25)
data2=data.loc[(data['date']>=start2) & (data['date']<end2)]

# part3 is the rest data that is not in part1 and part2
data3=data.loc[~(data['date'].isin(data1['date']) | data['date'].isin(data2['date']))]

# save to pkl
data1.to_pickle('ch11_data1.pkl')
data2.to_pickle('ch11_data2.pkl')
data3.to_pickle('c11_data3.pkl')